In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install -q convokit
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

#DATASET

to include glove file in your kernel, simple click 'input files' at the top of the notebook, and search 'glove' in the 'datasets' section.

In [ ]:
from convokit import Corpus, download
corpus = Corpus(filename=download("friends-corpus"))
import tensorflow as tf 
corpus.print_summary_stats()
data2 = corpus.get_utterances_dataframe()
data1 = data2[['text', 'speaker']]
data1 = data1.reset_index()
data1 = data1.drop('id',axis=1)
print(data1)

Dataset already exists at /root/.convokit/downloads/friends-corpus
Number of Speakers: 700
Number of Utterances: 67373
Number of Conversations: 3107
                                                    text          speaker
0      There's nothing to tell! He's just some guy I ...    Monica Geller
1      C'mon, you're going out with the guy! There's ...   Joey Tribbiani
2      All right Joey, be nice. So does he have a hum...    Chandler Bing
3                               Wait, does he eat chalk?    Phoebe Buffay
4                                                         TRANSCRIPT_NOTE
...                                                  ...              ...
67368                            Oh, it's gonna be okay.    Chandler Bing
67369  Do you guys have to go to the new house right ...     Rachel Green
67370                                  We got some time.    Monica Geller
67371                   Okay, should we get some coffee?     Rachel Green
67372                                

In [ ]:
#data1['list'] = data1[data1.columns[-1:]].values.tolist()
new_df = data1[['text', 'speaker']].copy()
new_df.head()
#print(new_df.iloc[i][1])

,text,speaker
0,There's nothing to tell! He's just some guy I ...,Monica Geller
1,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani
2,"All right Joey, be nice. So does he have a hum...",Chandler Bing
3,"Wait, does he eat chalk?",Phoebe Buffay
4,,TRANSCRIPT_NOTE


In [ ]:
# 0 = chandler
# 1 = Rachel
# 2 = Ross
# 3 = Monica
# 4 = Phoebe
# 5 = Joey


for i in range(len(new_df)):
    #item = (new_df.iloc[i][-1]).pop(0)
    if(new_df.iloc[i][-1] == 'Chandler Bing'):
        (new_df.loc[i][-1]) = np.array([1,0,0,0,0,0])
    elif(new_df.iloc[i][-1] == 'Rachel Green'):
        (new_df.loc[i][-1]) = np.array([0,1,0,0,0,0])
    elif(new_df.iloc[i][-1] == 'Ross Geller'):
        (new_df.loc[i][-1]) = np.array([0,0,1,0,0,0])
    elif(new_df.iloc[i][-1] == 'Monica Geller'):
        (new_df.loc[i][-1]) = np.array([0,0,0,1,0,0])
    elif(new_df.iloc[i][-1] == 'Phoebe Buffay'):
        (new_df.loc[i][-1]) = np.array([0,0,0,0,1,0])
    elif(new_df.iloc[i][-1] == 'Joey Tribbiani'):
        (new_df.loc[i][-1]) = np.array([0,0,0,0,0,1])
    else:
        (new_df.loc[i][-1]) = 'o'

new_df.sample(frac=1) 
new_df.head()

,text,speaker
0,There's nothing to tell! He's just some guy I ...,"[0, 0, 0, 1, 0, 0]"
1,"C'mon, you're going out with the guy! There's ...","[0, 0, 0, 0, 0, 1]"
2,"All right Joey, be nice. So does he have a hum...","[1, 0, 0, 0, 0, 0]"
3,"Wait, does he eat chalk?","[0, 0, 0, 0, 1, 0]"
4,,o


In [ ]:
new_df = new_df[new_df['speaker'] != 'o']

#new_df["list"] = pd.to_numeric(new_df["list"])

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:57: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [ ]:

EMBEDDING_FILE='glove.6B.100d.txt'
#TRAIN_DATA_FILE=f'friends-personality.csv'
#TEST_DATA_FILE=f'{path}{comp}test.csv'

In [ ]:
#set parameters
embed_size = 100 # how big is each word vector
max_features = 5414 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 300 # max number of words in a comment to use

In [ ]:
#read datasets
train = new_df
print(train)

                                                    text             speaker
0      There's nothing to tell! He's just some guy I ...  [0, 0, 0, 1, 0, 0]
1      C'mon, you're going out with the guy! There's ...  [0, 0, 0, 0, 0, 1]
2      All right Joey, be nice. So does he have a hum...  [1, 0, 0, 0, 0, 0]
3                               Wait, does he eat chalk?  [0, 0, 0, 0, 1, 0]
5      Just, 'cause, I don't want her to go through w...  [0, 0, 0, 0, 1, 0]
...                                                  ...                 ...
67368                            Oh, it's gonna be okay.  [1, 0, 0, 0, 0, 0]
67369  Do you guys have to go to the new house right ...  [0, 1, 0, 0, 0, 0]
67370                                  We got some time.  [0, 0, 0, 1, 0, 0]
67371                   Okay, should we get some coffee?  [0, 1, 0, 0, 0, 0]
67372                                       Sure. Where?  [1, 0, 0, 0, 0, 0]

[51312 rows x 2 columns]


#PRE PROCESSAMENTO 

In [ ]:
list_sentences_train = train["text"].fillna("_na_").values
list_classes = ["speaker"]
y = train[list_classes].values

#list_sentences_test = test["text"].fillna("_na_").values

In [ ]:
type(train[list_classes])


pandas.core.frame.DataFrame

In [ ]:
type(new_df)


pandas.core.frame.DataFrame

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
#list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
#X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

#WORD EMBBEDING

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.004451992, 0.4081574)

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
print(embed_size, max_features)

100 5414


#TREINO E TESTE

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(y)


tf.Tensor(
[[[0. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0. 0.]]

 ...

 [[0. 0. 0. 1. 0. 0.]]

 [[0. 1. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0.]]], shape=(51312, 1, 6), dtype=float32)


In [ ]:
 y=tf.convert_to_tensor(list(y), dtype=tf.float32)
 print(y)

tf.Tensor(
[[[0. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0. 0.]]

 ...

 [[0. 0. 0. 1. 0. 0.]]

 [[0. 1. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0.]]], shape=(51312, 1, 6), dtype=float32)


In [ ]:
y = tf.reshape(y, (51312, 6))
print(y)

tf.Tensor(
[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]], shape=(51312, 6), dtype=float32)


In [ ]:
model.fit(X_t, y, batch_size=32, epochs=3, validation_split=0.1);

Epoch 1/3
1444/1444 [==============================] - 1970s 1s/step - loss: 0.4521 - accuracy: 0.2090 - val_loss: 0.4467 - val_accuracy: 0.2473
Epoch 2/3
1444/1444 [==============================] - 2023s 1s/step - loss: 0.4371 - accuracy: 0.2680 - val_loss: 0.4410 - val_accuracy: 0.2603
Epoch 3/3
1444/1444 [==============================] - 2016s 1s/step - loss: 0.4243 - accuracy: 0.3071 - val_loss: 0.4407 - val_accuracy: 0.2703


#PARTE 2

In [ ]:
model.save_weights('./modelo/')


In [ ]:
TRAIN_DATA_FILE=f'friends-personality.csv'
train = pd.read_csv(TRAIN_DATA_FILE)
train['zero'] = 0 
print(train)

     Unnamed: 0    scene_id  ... cNEU zero
0             0  01_e01_c01  ...    1    0
1             1  01_e01_c02  ...    1    0
2             2  01_e01_c04  ...    1    0
3             3  01_e01_c05  ...    0    0
4             4  01_e01_c08  ...    1    0
..          ...         ...  ...  ...  ...
706         706  04_e24_c18  ...    0    0
707         707  04_e24_c19  ...    0    0
708         708  04_e24_c19  ...    0    0
709         709  04_e24_c20  ...    1    0
710         710  04_e24_c24  ...    1    0

[711 rows x 10 columns]


In [ ]:
list_sentences_train = train["text"].fillna("_na_").values

list_classes = ["cAGR", "cCON", "cEXT", "cOPN", "cNEU", "zero"]
y = train[list_classes].values
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
#list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
#X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.004451992, 0.4081574)

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
print(embed_size, max_features)

100 5414


In [ ]:


inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.load_weights('./modelo/')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(y)
type(y)

[[1 1 0 0 1 0]
 [1 0 1 1 1 0]
 [0 0 0 0 1 0]
 ...
 [1 0 1 1 0 0]
 [0 1 1 0 1 0]
 [1 0 0 1 1 0]]


numpy.ndarray

In [ ]:
model.fit(X_t, y, batch_size=128, epochs=50, validation_split=0.1);



Epoch 1/50
5/5 [==============================] - 12s 2s/step - loss: 0.5583 - accuracy: 0.2285 - val_loss: 0.5686 - val_accuracy: 0.1389
Epoch 2/50
5/5 [==============================] - 12s 2s/step - loss: 0.5544 - accuracy: 0.2207 - val_loss: 0.5697 - val_accuracy: 0.1250
Epoch 3/50
5/5 [==============================] - 12s 2s/step - loss: 0.5540 - accuracy: 0.1659 - val_loss: 0.5695 - val_accuracy: 0.1250
Epoch 4/50
5/5 [==============================] - 12s 2s/step - loss: 0.5513 - accuracy: 0.2034 - val_loss: 0.5685 - val_accuracy: 0.1528
Epoch 5/50
5/5 [==============================] - 12s 2s/step - loss: 0.5519 - accuracy: 0.2473 - val_loss: 0.5681 - val_accuracy: 0.1944
Epoch 6/50
5/5 [==============================] - 12s 2s/step - loss: 0.5459 - accuracy: 0.2347 - val_loss: 0.5692 - val_accuracy: 0.1389
Epoch 7/50
5/5 [==============================] - 12s 2s/step - loss: 0.5485 - accuracy: 0.2520 - val_loss: 0.5693 - val_accuracy: 0.1806
Epoch 8/50
5/5 [==================